#  Install TiRex package


In [1]:
!pip install 'tirex-ts[notebooks,gluonts,hfdataset,classification]' -q
!pip install aeon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


In [2]:
!pip install 'git+https://github.com/NX-AI/tirex.git#egg=tirex-ts[classification]'


DEPRECATION: git+https://github.com/NX-AI/tirex.git#egg=tirex-ts[classification] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NX-AI/tirex.git to /tmp/pip-install-rds1q0sc/tirex-ts_b27c0b2be8384f01be3e1a0d32bac956
  Running command git clone --filter=blob:none --quiet https://github.com/NX-AI/tirex.git /tmp/pip-install-rds1q0sc/tirex-ts_b27c0b2be8384f01be3e1a0d32bac956
  Resolved https://github.com/NX-AI/tirex.git to commit c54719aab2807c6dbd6a367896ce34f5306795eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Load Example Data

In [4]:
import torch
from aeon.datasets import load_italy_power_demand
from sklearn.preprocessing import LabelEncoder

# Load train data
train_X, train_y = load_italy_power_demand(split="train")

# Load test data
test_X, test_y = load_italy_power_demand(split="test")

# Encode string labels -> integers
label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
test_y = label_encoder.transform(test_y)

# Convert to torch tensors
train_X = torch.tensor(train_X, dtype=torch.float32)
test_X = torch.tensor(test_X, dtype=torch.float32)

train_y = torch.tensor(train_y, dtype=torch.long)
test_y = torch.tensor(test_y, dtype=torch.long)

print(train_X.shape, train_y.shape)
print(test_X.shape, test_y.shape)

torch.Size([67, 1, 24]) torch.Size([67])
torch.Size([1029, 1, 24]) torch.Size([1029])


In [5]:
train_X[:5]

tensor([[[-0.7105, -1.1833, -1.3724, -1.5931, -1.4670, -1.3724, -1.0888,
           0.0460,  0.9285,  1.0861,  1.2753,  0.9601,  0.6133,  0.0144,
          -0.6475, -0.2692, -0.2062,  0.6133,  1.3698,  1.4644,  1.0546,
           0.5818,  0.1720, -0.2692]],

        [[-0.9930, -1.4268, -1.5799, -1.6054, -1.6309, -1.3758, -1.0185,
          -0.3551,  0.7166,  1.2014,  1.1248,  1.0483,  0.7931,  0.4614,
           0.4869,  0.5635,  0.6145,  0.3083,  0.2573,  1.0993,  1.0483,
           0.6911, -0.0489, -0.3806]],

        [[ 1.3191,  0.5698,  0.1951, -0.0859, -0.1795, -0.2732, -0.0859,
          -1.3971, -1.1161, -0.7415,  0.0078, -0.0859,  0.0078, -0.4605,
          -0.5542, -0.7415, -0.7415, -0.7415, -1.1161, -0.4605,  0.4761,
           2.3493,  2.2557,  1.6001]],

        [[-0.8124, -1.1576, -1.4164, -1.5314, -1.5027, -1.4164, -1.6465,
          -0.4673,  0.6543,  1.0569,  1.3445,  1.2007,  0.9994,  0.6543,
           0.7405,  0.8843,  0.8843,  0.6830,  0.6255,  0.4242, -0.0072,
    

In [6]:
train_y[:5]

tensor([0, 0, 1, 1, 0])

# Initialize Embedding model and Classification Head

## 1. Random Forest Classifier

In [7]:
from tirex.models.classification import TirexRFClassifier

classifier = TirexRFClassifier(n_estimators=50, max_depth=10, random_state=42, batch_size=256)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.ckpt:   0%|          | 0.00/141M [00:00<?, ?B/s]

### Fit Model

In [8]:
fit_meta = classifier.fit((train_X, train_y))

### Prediction results

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

pred_y = classifier.predict(test_X)
pred_y = pred_y.cpu().numpy()

print("Accuracy " + str(accuracy_score(test_y, pred_y)))
print("F1 Score " + str(f1_score(test_y, pred_y, average="macro")))

## 2. Linear Regression Classifier

In [ ]:
from tirex.models.classification import TirexLinearClassifier

classifier_2 = TirexLinearClassifier(data_augmentation=False, compile=False)

In [ ]:
fit_meta_2 = classifier_2.fit((train_X, train_y))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

pred_y = classifier_2.predict(test_X)
pred_y = pred_y.cpu().numpy()

print("Accuracy " + str(accuracy_score(test_y, pred_y)))
print("F1 Score " + str(f1_score(test_y, pred_y, average="macro")))

## 3. Gradient Boosting Classifier

In [ ]:
from tirex.models.classification import TirexGBMClassifier

classifier_3 = TirexGBMClassifier(data_augmentation=False, compile=False)

In [ ]:
fit_meta_3 = classifier_3.fit((train_X, train_y))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

pred_y = classifier_3.predict(test_X)
pred_y = pred_y.cpu().numpy()

print("Accuracy " + str(accuracy_score(test_y, pred_y)))
print("F1 Score " + str(f1_score(test_y, pred_y, average="macro")))